In [1]:
import dice_ml
from dice_ml import Dice
from dice_ml.utils.exception import UserConfigValidationException

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris, fetch_california_housing

from src.transition_system import transition_system, indexs_for_window, list_to_str
from src.function_store import StoreTestRun, extract_algo_name, generate_cfe, get_case_id, prepare_df_for_ml, \
    activity_n_resources, get_test_cases, get_prefix_of_activities, validate_transition

import pandas as pd
import os
import pickle
import random
import subprocess
from IPython.display import display
from math import ceil
from time import time
import warnings
from wrapt_timeout_decorator import timeout

# from multiprocessing import Process
# from timeout_decorator import timeout, TimeoutError
# import signal
from typing import Tuple, Any, List, Union
from collections import Counter
import utils
from time import sleep
pd.options.display.max_columns = 50
pd.set_option('display.max_rows', 500)
if "src" in os.getcwd():
    os.chdir("../")

# Suppress all warnings
warnings.filterwarnings("ignore")
%load_ext autoreload
%autoreload 2

# Dataset: Bank Account Closure

## Setup Variables

In [5]:
KPI = "activity_occurrence"  # activity_occurrence, ...
SECONDS_TO_HOURS = 60 * 60
SECONDS_TO_DAYS = 60 * 60 * 24
WINDOW_SIZE = 3
REDUCED_KPI_TIME = 90
TOTAL_CFS = 15                        # Number of CFs DiCE algorithm should produce
TRAIN_DATA_SIZE = 170_335             # 170_335
DICE_METHOD = "random"
RESULTS_FILE_PATH_N_NAME = "experiment_results/random-a01-local.csv"
# ToDo store the model configurations in the pkl file as well and retrieve them from here
proximity_weight = 0.2
sparsity_weight = 0.2
diversity_weight = 5.0

case_id_name = 'REQUEST_ID'  # The case identifier column name.
# start_date_name = 'Change_Date+Time'  # Maybe change to start_et (start even time)
activity_column_name = "ACTIVITY"

## Load Data

In [6]:
data_dir = "./preprocessed_datasets/"
train_dataset_file = "bank_acc_train.csv"
test_dataset_file = "bank_acc_test.csv"
test_pickle_dataset_file = "bank_acc-test.pkl"
df = pd.read_csv("./data/completed.csv")  # Use full dataset for transition systens
df_train = pd.read_csv(os.path.join(data_dir, train_dataset_file))
df_test = pd.read_csv(os.path.join(data_dir, test_dataset_file))

### Basic Preprocessing of the dataset

In [7]:
df = df.fillna("missing")
df_train = df_train[:TRAIN_DATA_SIZE]  # 31_066
# df_test = df_test[: 19_041]
print(f"Rows in df_train: {len(df_train):,}")
print(f"Rows in df_test: {len(df_test):,}")
# df_train.info()

Rows in df_train: 170,335
Rows in df_test: 6,391


### Helper functions

In [8]:
# valid_resources = activity_n_resources(df_train, ["ACTIVITY", "Involved_ST_Function_Div", "Involved_Org_line_3", "Involved_ST"])
resource_columns_to_validate = [activity_column_name, 'CE_UO', 'ROLE']
valid_resources = activity_n_resources(df, resource_columns_to_validate, threshold_percentage=90)
# len(valid_resources)

In [9]:
# # === Analysis of all the columns in the dataset
# variable_type_analysis(df, case_id_name=case_id_name, activity_name=activity_column_name)

### Prepare the Test Dataset

In [10]:
# === Unpickle the Standard test-set. To standardize the test across different parameters.
test_cases = get_test_cases(None, None, load_dataset=True, path_and_filename=os.path.join(data_dir, test_pickle_dataset_file))
print(f"Number of cases in the test set: {len(test_cases):,}")

# # === Pickle dataset for comparison of different methods
# with open(os.path.join(data_dir, test_pickle_dataset_file), 'wb') as file:
#     pickle.dump(test_cases, file)

Number of cases in the test set: 1,096


In [11]:
# ### Features that can vary
# option 1:
# cols_to_vary = [col for col in df_train.columns if col[0] == '#']

cols_to_vary = ["ACTIVITY", "CE_UO", "ROLE"]

outcome_name = "Back-Office Adjustment Requested"

X_train, y_train = prepare_df_for_ml(df_train, case_id_name, outcome_name, columns_to_remove=["START_DATE", "END_DATE", "time_remaining"])
# X_test, y_test = prepare_df_for_ml(df_test, outcome_name)
continuous_features = ["time_from_first", "time_from_previous_et", "time_from_midnight", "activity_duration", '# ACTIVITY=Service closure Request with network responsibility',
                    '# ACTIVITY=Service closure Request with BO responsibility', '# ACTIVITY=Pending Request for Reservation Closure', '# ACTIVITY=Pending Liquidation Request',
                    '# ACTIVITY=Request completed with account closure', '# ACTIVITY=Request created', '# ACTIVITY=Authorization Requested',
                    '# ACTIVITY=Evaluating Request (NO registered letter)', '# ACTIVITY=Network Adjustment Requested', '# ACTIVITY=Pending Request for acquittance of heirs',
                    '# ACTIVITY=Request deleted', '# ACTIVITY=Back-Office Adjustment Requested', '# ACTIVITY=Evaluating Request (WITH registered letter)',
                    '# ACTIVITY=Request completed with customer recovery', '# ACTIVITY=Pending Request for Network Information',]
categorical_features = ["CLOSURE_TYPE", "CLOSURE_REASON", "ACTIVITY", "CE_UO", "ROLE", "weekday"]

# We create the preprocessing pipelines for both numeric and categorical data.
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

transformations = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, continuous_features),
        ('cat', categorical_transformer, categorical_features)])

# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
clf = Pipeline(steps=[('preprocessor', transformations),
                           ('classifier', RandomForestClassifier(n_jobs=7))])
model = clf.fit(X_train, y_train)

# ## Create DiCE model
data_model = dice_ml.Data(dataframe=pd.concat([X_train, y_train], axis="columns"),
                      continuous_features=continuous_features,
                      outcome_name=outcome_name)

# We provide the type of model as a parameter (model_type)
ml_backend = dice_ml.Model(model=model, backend="sklearn", model_type='classifier')
method = DICE_METHOD  # genetic, kdtree, random
explainer = Dice(data_model, ml_backend, method=method)  # Categorical features do not support features_weights argument in generate_counterfactuals()
# exp_genetic_iris = Dice(d_iris, m_iris, method="kdtree")

## Experiment with Single Queries

In [14]:
sidx = 612
eidx = 622

df_train[sidx: eidx][[ "REQUEST_ID", "CLOSURE_TYPE", "CLOSURE_REASON", "ACTIVITY", "CE_UO", "ROLE", "Back-Office Adjustment Requested"] ]
# df_train[sidx: eidx]

,REQUEST_ID,CLOSURE_TYPE,CLOSURE_REASON,ACTIVITY,CE_UO,ROLE,Back-Office Adjustment Requested
612,20182001337,Bank Recess,1 - Client lost,Service closure Request with BO responsibility,BOC,BACK-OFFICE,0
613,20182001337,Bank Recess,1 - Client lost,Pending Request for Reservation Closure,BOC,BACK-OFFICE,0
614,20182001337,Bank Recess,1 - Client lost,Pending Liquidation Request,BOC,BACK-OFFICE,0
615,20182001491,Client Recess,1 - Client lost,Request created,00877,APPLICANT,1
616,20182001491,Client Recess,1 - Client lost,Evaluating Request (NO registered letter),00037,DIRECTOR,1
617,20182001491,Client Recess,1 - Client lost,Service closure Request with network responsib...,00037,APPLICANT,1
618,20182001491,Client Recess,1 - Client lost,Service closure Request with BO responsibility,BOC,BACK-OFFICE,1
619,20182001491,Client Recess,1 - Client lost,Pending Request for Reservation Closure,BOC,BACK-OFFICE,1
620,20182001491,Client Recess,1 - Client lost,Back-Office Adjustment Requested,BOC,BACK-OFFICE,0
621,20182001491,Client Recess,1 - Client lost,Pending Request for Reservation Closure,BOC,BACK-OFFICE,0


In [15]:
sidx = 619
eidx = 620

query_instances = X_train[sidx: eidx]
cfe = explainer.generate_counterfactuals(query_instances, total_CFs=15, desired_class="opposite", features_to_vary=cols_to_vary,
                                                permitted_range = {"ACTIVITY": ['Service closure Request with network responsibility',
                                                                            'Service closure Request with BO responsibility',
                                                                            'Pending Request for Reservation Closure', 'Pending Liquidation Request',
                                                                            'Request created','Authorization Requested', 'Evaluating Request (NO registered letter)',
                                                                            'Network Adjustment Requested', 'Evaluating Request (WITH registered letter)',
                                                                            'Pending Request for Network Information']})  # 'Back-Office Adjustment Requested'

cfe.visualize_as_dataframe(show_only_changes=True)

100%|██████████| 1/1 [00:01<00:00,  1.18s/it]

Query instance (original outcome : 1)


,CLOSURE_TYPE,CLOSURE_REASON,ACTIVITY,CE_UO,ROLE,time_from_first,time_from_previous_et,time_from_midnight,weekday,activity_duration,# ACTIVITY=Service closure Request with network responsibility,# ACTIVITY=Service closure Request with BO responsibility,# ACTIVITY=Pending Request for Reservation Closure,# ACTIVITY=Pending Liquidation Request,# ACTIVITY=Request completed with account closure,# ACTIVITY=Request created,# ACTIVITY=Authorization Requested,# ACTIVITY=Evaluating Request (NO registered letter),# ACTIVITY=Network Adjustment Requested,# ACTIVITY=Pending Request for acquittance of heirs,# ACTIVITY=Request deleted,# ACTIVITY=Back-Office Adjustment Requested,# ACTIVITY=Evaluating Request (WITH registered letter),# ACTIVITY=Request completed with customer recovery,# ACTIVITY=Pending Request for Network Information,Back-Office Adjustment Requested
0,Client Recess,1 - Client lost,Pending Request for Reservation Closure,BOC,BACK-OFFICE,26354849,810,45978,Friday,64,1,1,0,0,0,1,0,1,0,0,0,0,0,0,0,1



Diverse Counterfactual set (new outcome: 0.0)


,CLOSURE_TYPE,CLOSURE_REASON,ACTIVITY,CE_UO,ROLE,time_from_first,time_from_previous_et,time_from_midnight,weekday,activity_duration,# ACTIVITY=Service closure Request with network responsibility,# ACTIVITY=Service closure Request with BO responsibility,# ACTIVITY=Pending Request for Reservation Closure,# ACTIVITY=Pending Liquidation Request,# ACTIVITY=Request completed with account closure,# ACTIVITY=Request created,# ACTIVITY=Authorization Requested,# ACTIVITY=Evaluating Request (NO registered letter),# ACTIVITY=Network Adjustment Requested,# ACTIVITY=Pending Request for acquittance of heirs,# ACTIVITY=Request deleted,# ACTIVITY=Back-Office Adjustment Requested,# ACTIVITY=Evaluating Request (WITH registered letter),# ACTIVITY=Request completed with customer recovery,# ACTIVITY=Pending Request for Network Information,Back-Office Adjustment Requested
0,-,-,-,00590,APPLICANT,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,0
1,-,-,Pending Liquidation Request,00707,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,0
2,-,-,-,00048,APPLICANT,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,0
3,-,-,Service closure Request with BO responsibility,00211,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,0
4,-,-,Pending Liquidation Request,00346,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,0
5,-,-,-,00446,APPLICANT,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,0
6,-,-,-,00270,APPLICANT,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,0
7,-,-,Authorization Requested,00502,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,0
8,-,-,Authorization Requested,00366,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,0
9,-,-,-,00654,APPLICANT,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,0


## Apply Transition System Validation to the above Single Query Results

In [16]:
## Apply Transition System Validation to the above Single Query Results
# === Load the Transition Graph
_, transition_graph = transition_system(df, case_id_name=case_id_name, activity_column_name=activity_column_name, window_size=WINDOW_SIZE)

### Check Feature Importance score

In [17]:
# imp = explainer.local_feature_importance(query_instances, total_CFs=10, desired_range=[0, total_time_upper_bound], features_to_vary=cols_to_vary)
# imp.local_importance

## Experiment with Multiple Queries

In [18]:
state_obj = StoreTestRun(save_load_path=RESULTS_FILE_PATH_N_NAME)
save_load_path = state_obj.get_save_load_path()

if os.path.exists(save_load_path):
    state_obj.load_state()
    cases_done = state_obj.run_state["cases_done"]
else:
    cases_done = 0

state_obj.run_state["cases_done"]

0

In [ ]:
# %%capture
start_from_case = state_obj.run_state["cases_done"]
for df_test_trace in test_cases[cases_done:]:
    test_trace_start = time()
    # print("Case number:", cases_done)
    # print(f"Case number is state_obj: {state_obj.run_state['cases_done']}")
    query_case_id = get_case_id(df_test_trace, case_id_name)

    if 0 < len(df_test_trace) <= 2:
        print("too small", cases_done, query_case_id)
        result_value = query_case_id
        state_obj.add_cfe_to_results( ("cases_too_small", result_value) )
        cases_stored = state_obj.save_state()
        cases_done += 1
        continue

    X_test, y_test = prepare_df_for_ml(df_test_trace, case_id_name, outcome_name, columns_to_remove=["START_DATE", "END_DATE", "time_remaining"])

    # Check if y_test is 0 then don't generate CFE
    if y_test.iloc[-1] == 0:
        result_value = query_case_id
        state_obj.add_cfe_to_results(("cases_zero_in_y", result_value))
        cases_stored = state_obj.save_state()
        cases_done += 1
        continue

    # Access the last row of the truncated trace to replicate the behavior of a running trace
    query_instances = X_test.iloc[-1:]

    try:
        cfe = generate_cfe(explainer, query_instances, None, total_cfs=TOTAL_CFS, features_to_vary=cols_to_vary, kpi=KPI,
                           proximity_weight=proximity_weight, sparsity_weight=sparsity_weight, diversity_weight=diversity_weight )

        result_value = (query_case_id, cfe)
        state_obj.add_cfe_to_results(("cfe_before_validation", result_value))  # save after cfe validation

        prefix_of_activities = get_prefix_of_activities(df_single_trace=df_test_trace, window_size=WINDOW_SIZE, activity_column_name=activity_column_name,)
        cfe_df = validate_transition(cfe, prefix_of_activities=prefix_of_activities, transition_graph=transition_graph, valid_resources=valid_resources,
                                     activity_column_name=activity_column_name, resource_columns_to_validate=resource_columns_to_validate)
        if len(cfe_df) > 0:
            result_value = (query_case_id, cfe_df)
            state_obj.add_cfe_to_results(("cfe_after_validation", result_value))

        cases_stored = state_obj.save_state()

    except UserConfigValidationException:
        result_value = query_case_id
        state_obj.add_cfe_to_results(("cfe_not_found", result_value))
        cases_stored = state_obj.save_state()
    except TimeoutError as err:  # When function takes too long
        result_value = query_case_id
        print("TimeoutError caught:", err)
        state_obj.add_cfe_to_results(("cfe_not_found", result_value))
        cases_stored = state_obj.save_state()
    except ValueError:
        # print(f"Includes feature not found in training data: {get_case_id(df_test_trace)}")
        result_value = query_case_id
        state_obj.add_cfe_to_results(("cases_includes_new_data", result_value))
        cases_stored = state_obj.save_state()
    # This error is seen occurring on when running lots of loops on the server
    except AttributeError as e:
        print("AttributeError caught:", e)
        state_obj.add_cfe_to_results(("exceptions", query_case_id))
        cases_stored = state_obj.save_state()
    # except Exception as err:
    #     print(f"Broadest Exception handler invoked", err)
    #     state_obj.add_cfe_to_results(("exceptions", query_case_id))
    #     cases_stored = state_obj.save_state()

    cases_done += 1
    # Just for a sanity check
    print(f"Time it took: { round( ((time() - test_trace_start) / 60 ), 3) }")
    assert cases_done == cases_stored
    if cases_done >= 10:
        break

In [22]:
state_obj.get_run_state_df()

,cfe_before_validation,cfe_after_validation,cfe_not_found,cases_includes_new_data,cases_too_small,cases_zero_in_y,exceptions,cases_done
0,10,3,0,0,0,0,0,10


# Results:

### View Valid cases and their Counterfactual Examples (CFEs)
You can load saved configs and run them independently of the above code

In [80]:
view_from_saved_file = True  # Set this option true and specify the different file name.
pickle_file = "random-a08.pkl"
if view_from_saved_file:
    if not os.path.exists( f"./experiment_results/{pickle_file}"):
        result = subprocess.run(['scp', f'labnum08:git_repos/explainable-prescriptive-analytics/experiment_results/{pickle_file}', 'experiment_results/'], capture_output=True, text=True)

        # return code of 0 means the command executed successfully
        if result.returncode != 0:
            print("There is an Error in the command")
        else:
            print("successful")
    else:
        print(f"File already exists")

    RESULTS_FILE_PATH_N_NAME = f"experiment_results/{pickle_file.split('.')[0]}.csv"
else:
    print(f"Using results from file used in the above code")

File already exists


In [81]:
if view_from_saved_file:
    RESULTS_FILE_PATH_N_NAME = f"experiment_results/{pickle_file.split('.')[0]}.csv"
state_obj = StoreTestRun(save_load_path=RESULTS_FILE_PATH_N_NAME)
save_load_path = state_obj.get_save_load_path()

if os.path.exists(save_load_path):
    state_obj.load_state()
    cases_done = state_obj.run_state["cases_done"]
else:
    cases_done = 0

print("Cases tested: ", state_obj.run_state["cases_done"])
def print_results():
    for case_id, cfe_df in state_obj.run_state["cfe_after_validation"]:
        yield case_id, cfe_df

generator = print_results()

Cases tested:  1096


In [82]:
case_id, cfe_df = next(generator)

print(f"Original Test Case:")
# === Find test_case with case_id
for df_test_trace in test_cases:
    if get_case_id(df_test_trace, case_id_name) == case_id:
        display(df_test_trace)
        df_test_trace.to_csv("temp.csv", index=False)

print(f"Counterfactuals for the last row:")
cfe_df.to_csv("cfe_temp.csv", index=False)
cfe_df

Original Test Case:


,REQUEST_ID,START_DATE,CLOSURE_TYPE,CLOSURE_REASON,ACTIVITY,END_DATE,CE_UO,ROLE,time_from_first,time_from_previous_et,time_from_midnight,weekday,activity_duration,time_remaining,# ACTIVITY=Service closure Request with network responsibility,# ACTIVITY=Service closure Request with BO responsibility,# ACTIVITY=Pending Request for Reservation Closure,# ACTIVITY=Pending Liquidation Request,# ACTIVITY=Request completed with account closure,# ACTIVITY=Request created,# ACTIVITY=Authorization Requested,# ACTIVITY=Evaluating Request (NO registered letter),# ACTIVITY=Network Adjustment Requested,# ACTIVITY=Pending Request for acquittance of heirs,# ACTIVITY=Request deleted,# ACTIVITY=Back-Office Adjustment Requested,# ACTIVITY=Evaluating Request (WITH registered letter),# ACTIVITY=Request completed with customer recovery,# ACTIVITY=Pending Request for Network Information,Back-Office Adjustment Requested
0,20183001089,1520245359,Bank Recess,1 - Client lost,Request created,1520245395,SB24,APPLICANT,0,0,37359,Monday,36,16585730,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,20183001089,1520245395,Bank Recess,1 - Client lost,Service closure Request with network responsib...,1524239518,missing,APPLICANT,36,36,37395,Monday,3994123,16585694,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1
2,20183001089,1524239518,Bank Recess,1 - Client lost,Authorization Requested,1524239835,00443,DIRECTOR,3994159,3994123,57118,Friday,317,12591571,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1
3,20183001089,1524239835,Bank Recess,1 - Client lost,Network Adjustment Requested,1532446170,00443,APPLICANT,3994476,317,57435,Friday,8206335,12591254,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1
4,20183001089,1532446170,Bank Recess,1 - Client lost,Authorization Requested,1532446231,00443,DIRECTOR,12200811,8206335,55770,Tuesday,61,4384919,1,0,0,0,0,1,1,0,1,0,0,0,0,0,0,1


Counterfactuals for the last row:


,CLOSURE_TYPE,CLOSURE_REASON,ACTIVITY,CE_UO,ROLE,time_from_first,time_from_previous_et,time_from_midnight,weekday,activity_duration,# ACTIVITY=Service closure Request with network responsibility,# ACTIVITY=Service closure Request with BO responsibility,# ACTIVITY=Pending Request for Reservation Closure,# ACTIVITY=Pending Liquidation Request,# ACTIVITY=Request completed with account closure,# ACTIVITY=Request created,# ACTIVITY=Authorization Requested,# ACTIVITY=Evaluating Request (NO registered letter),# ACTIVITY=Network Adjustment Requested,# ACTIVITY=Pending Request for acquittance of heirs,# ACTIVITY=Request deleted,# ACTIVITY=Back-Office Adjustment Requested,# ACTIVITY=Evaluating Request (WITH registered letter),# ACTIVITY=Request completed with customer recovery,# ACTIVITY=Pending Request for Network Information,Back-Office Adjustment Requested
0,Bank Recess,1 - Client lost,Authorization Requested,00581,DIRECTOR,12200811,8206335,55770,Tuesday,61,1,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0


In [83]:
### From case_id view test trace/ case and the full trace/ case

In [84]:
state_obj.run_state["cfe_not_found"][:10]

[20178008599,
 20181002993,
 20182009564,
 20182011180,
 20184005244,
 20184005305,
 20184006674,
 20184007422,
 20185001269,
 20185005324]

In [102]:
search_case_id = 20184005305
for df_test_trace in test_cases:
    query_case_id = get_case_id(df_test_trace, case_id_name)
    if query_case_id == search_case_id:
        df_test_trace_temp = df_test_trace
        display(df_test_trace_temp)

# only print part of the trace that is not present in the df_test_trace
df_trace = df_test[df_test[case_id_name] == search_case_id]

display(df_trace.iloc[len(df_test_trace_temp) - 1:])  # -1 gives overlap of 1 row in the df_trace print
# display(df_trace)

,REQUEST_ID,START_DATE,CLOSURE_TYPE,CLOSURE_REASON,ACTIVITY,END_DATE,CE_UO,ROLE,time_from_first,time_from_previous_et,time_from_midnight,weekday,activity_duration,time_remaining,# ACTIVITY=Service closure Request with network responsibility,# ACTIVITY=Service closure Request with BO responsibility,# ACTIVITY=Pending Request for Reservation Closure,# ACTIVITY=Pending Liquidation Request,# ACTIVITY=Request completed with account closure,# ACTIVITY=Request created,# ACTIVITY=Authorization Requested,# ACTIVITY=Evaluating Request (NO registered letter),# ACTIVITY=Network Adjustment Requested,# ACTIVITY=Pending Request for acquittance of heirs,# ACTIVITY=Request deleted,# ACTIVITY=Back-Office Adjustment Requested,# ACTIVITY=Evaluating Request (WITH registered letter),# ACTIVITY=Request completed with customer recovery,# ACTIVITY=Pending Request for Network Information,Back-Office Adjustment Requested
0,20184005305,1523960091,Client Recess,1 - Client lost,Request created,1523960173,SB11,APPLICANT,0,0,36891,Tuesday,82,24194798,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,20184005305,1523960173,Client Recess,1 - Client lost,Evaluating Request (NO registered letter),1525078738,00294,DIRECTOR,82,82,36973,Tuesday,1118565,24194716,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1
2,20184005305,1547743086,Client Recess,1 - Client lost,Service closure Request with network responsib...,1547743112,00294,APPLICANT,23782995,23782913,59886,Thursday,26,411803,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1
3,20184005305,1547743112,Client Recess,1 - Client lost,Service closure Request with BO responsibility,1547827116,BOC,BACK-OFFICE,23783021,26,59912,Thursday,84004,411777,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1
4,20184005305,1547827116,Client Recess,1 - Client lost,Pending Request for Reservation Closure,1547827208,BOC,BACK-OFFICE,23867025,84004,57516,Friday,92,327773,1,1,0,0,0,1,0,1,0,0,0,0,0,0,0,1


,REQUEST_ID,START_DATE,CLOSURE_TYPE,CLOSURE_REASON,ACTIVITY,END_DATE,CE_UO,ROLE,time_from_first,time_from_previous_et,time_from_midnight,weekday,activity_duration,time_remaining,# ACTIVITY=Service closure Request with network responsibility,# ACTIVITY=Service closure Request with BO responsibility,# ACTIVITY=Pending Request for Reservation Closure,# ACTIVITY=Pending Liquidation Request,# ACTIVITY=Request completed with account closure,# ACTIVITY=Request created,# ACTIVITY=Authorization Requested,# ACTIVITY=Evaluating Request (NO registered letter),# ACTIVITY=Network Adjustment Requested,# ACTIVITY=Pending Request for acquittance of heirs,# ACTIVITY=Request deleted,# ACTIVITY=Back-Office Adjustment Requested,# ACTIVITY=Evaluating Request (WITH registered letter),# ACTIVITY=Request completed with customer recovery,# ACTIVITY=Pending Request for Network Information,Back-Office Adjustment Requested
50,20184005305,1547827116,Client Recess,1 - Client lost,Pending Request for Reservation Closure,1547827208,BOC,BACK-OFFICE,23867025,84004,57516,Friday,92,327773,1,1,0,0,0,1,0,1,0,0,0,0,0,0,0,1
51,20184005305,1547827208,Client Recess,1 - Client lost,Pending Liquidation Request,1548126085,BOC,BACK-OFFICE,23867117,92,57608,Friday,298877,327681,1,1,1,0,0,1,0,1,0,0,0,0,0,0,0,1
52,20184005305,1548126085,Client Recess,1 - Client lost,Back-Office Adjustment Requested,1548154889,BOC,BACK-OFFICE,24165994,298877,10885,Tuesday,28804,28804,1,1,1,1,0,1,0,1,0,0,0,0,0,0,0,0


### Run CFE on a case from df_test
This is to experiment and see if running the CFE generator can produce an output if we run on the same trace (which doesn't produce output on the prefix in the test_set)
but on a different prefix (or split point). Use `sidx` (start index) to make a split manually anywhere in the testset.

In [104]:
for i in range(51, 52):
    sidx = i  # 58
    eidx = sidx + 1
    try:
        X_test, y_test = prepare_df_for_ml(df_test, case_id_name, outcome_name, columns_to_remove=["START_DATE", "END_DATE", "time_remaining"])
        query_instances = X_test[sidx: eidx]
        cfe = explainer.generate_counterfactuals(query_instances, total_CFs=15, desired_class="opposite",  # desired_class=0, #
                                                        features_to_vary=cols_to_vary,
                                                        permitted_range = {"ACTIVITY": ['Service closure Request with network responsibility',
                                                                                    'Service closure Request with BO responsibility',
                                                                                    'Pending Request for Reservation Closure', 'Pending Liquidation Request',
                                                                                    'Request created','Authorization Requested', 'Evaluating Request (NO registered letter)',
                                                                                    'Network Adjustment Requested', 'Evaluating Request (WITH registered letter)',
                                                                                    'Pending Request for Network Information']})  # 'Back-Office Adjustment Requested'
    except UserConfigValidationException:
        print("Error on idx: ", i)

cfe.visualize_as_dataframe(show_only_changes=True)

100%|██████████| 1/1 [00:00<00:00,  1.22it/s]

Only 1 (required 15)  Diverse Counterfactuals found for the given configuration, perhaps try with different parameters... ; total time taken: 00 min 00 sec
Query instance (original outcome : 0)


,CLOSURE_TYPE,CLOSURE_REASON,ACTIVITY,CE_UO,ROLE,time_from_first,time_from_previous_et,time_from_midnight,weekday,activity_duration,# ACTIVITY=Service closure Request with network responsibility,# ACTIVITY=Service closure Request with BO responsibility,# ACTIVITY=Pending Request for Reservation Closure,# ACTIVITY=Pending Liquidation Request,# ACTIVITY=Request completed with account closure,# ACTIVITY=Request created,# ACTIVITY=Authorization Requested,# ACTIVITY=Evaluating Request (NO registered letter),# ACTIVITY=Network Adjustment Requested,# ACTIVITY=Pending Request for acquittance of heirs,# ACTIVITY=Request deleted,# ACTIVITY=Back-Office Adjustment Requested,# ACTIVITY=Evaluating Request (WITH registered letter),# ACTIVITY=Request completed with customer recovery,# ACTIVITY=Pending Request for Network Information,Back-Office Adjustment Requested
0,Client Recess,1 - Client lost,Pending Liquidation Request,BOC,BACK-OFFICE,23867117,92,57608,Friday,298877,1,1,1,0,0,1,0,1,0,0,0,0,0,0,0,0



Diverse Counterfactual set (new outcome: 1.0)


,CLOSURE_TYPE,CLOSURE_REASON,ACTIVITY,CE_UO,ROLE,time_from_first,time_from_previous_et,time_from_midnight,weekday,activity_duration,# ACTIVITY=Service closure Request with network responsibility,# ACTIVITY=Service closure Request with BO responsibility,# ACTIVITY=Pending Request for Reservation Closure,# ACTIVITY=Pending Liquidation Request,# ACTIVITY=Request completed with account closure,# ACTIVITY=Request created,# ACTIVITY=Authorization Requested,# ACTIVITY=Evaluating Request (NO registered letter),# ACTIVITY=Network Adjustment Requested,# ACTIVITY=Pending Request for acquittance of heirs,# ACTIVITY=Request deleted,# ACTIVITY=Back-Office Adjustment Requested,# ACTIVITY=Evaluating Request (WITH registered letter),# ACTIVITY=Request completed with customer recovery,# ACTIVITY=Pending Request for Network Information,Back-Office Adjustment Requested
0,-,-,-,SDCTCC,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,1


In [99]:
df_test[45:55]

,REQUEST_ID,START_DATE,CLOSURE_TYPE,CLOSURE_REASON,ACTIVITY,END_DATE,CE_UO,ROLE,time_from_first,time_from_previous_et,time_from_midnight,weekday,activity_duration,time_remaining,# ACTIVITY=Service closure Request with network responsibility,# ACTIVITY=Service closure Request with BO responsibility,# ACTIVITY=Pending Request for Reservation Closure,# ACTIVITY=Pending Liquidation Request,# ACTIVITY=Request completed with account closure,# ACTIVITY=Request created,# ACTIVITY=Authorization Requested,# ACTIVITY=Evaluating Request (NO registered letter),# ACTIVITY=Network Adjustment Requested,# ACTIVITY=Pending Request for acquittance of heirs,# ACTIVITY=Request deleted,# ACTIVITY=Back-Office Adjustment Requested,# ACTIVITY=Evaluating Request (WITH registered letter),# ACTIVITY=Request completed with customer recovery,# ACTIVITY=Pending Request for Network Information,Back-Office Adjustment Requested
45,20184005244,1536894288,Client Recess,1 - Client lost,Back-Office Adjustment Requested,1537173189,BOC,BACK-OFFICE,12937058,150065,11088,Friday,278901,278901,1,1,1,1,0,1,0,1,0,0,0,0,0,0,0,0
46,20184005305,1523960091,Client Recess,1 - Client lost,Request created,1523960173,SB11,APPLICANT,0,0,36891,Tuesday,82,24194798,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
47,20184005305,1523960173,Client Recess,1 - Client lost,Evaluating Request (NO registered letter),1525078738,00294,DIRECTOR,82,82,36973,Tuesday,1118565,24194716,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1
48,20184005305,1547743086,Client Recess,1 - Client lost,Service closure Request with network responsib...,1547743112,00294,APPLICANT,23782995,23782913,59886,Thursday,26,411803,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1
49,20184005305,1547743112,Client Recess,1 - Client lost,Service closure Request with BO responsibility,1547827116,BOC,BACK-OFFICE,23783021,26,59912,Thursday,84004,411777,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1
50,20184005305,1547827116,Client Recess,1 - Client lost,Pending Request for Reservation Closure,1547827208,BOC,BACK-OFFICE,23867025,84004,57516,Friday,92,327773,1,1,0,0,0,1,0,1,0,0,0,0,0,0,0,1
51,20184005305,1547827208,Client Recess,1 - Client lost,Pending Liquidation Request,1548126085,BOC,BACK-OFFICE,23867117,92,57608,Friday,298877,327681,1,1,1,0,0,1,0,1,0,0,0,0,0,0,0,1
52,20184005305,1548126085,Client Recess,1 - Client lost,Back-Office Adjustment Requested,1548154889,BOC,BACK-OFFICE,24165994,298877,10885,Tuesday,28804,28804,1,1,1,1,0,1,0,1,0,0,0,0,0,0,0,0
53,20184006674,1524155208,Client Recess,1 - Client lost,Request created,1524155220,00409,APPLICANT,0,0,59208,Thursday,12,15438013,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
54,20184006674,1524155220,Client Recess,1 - Client lost,Evaluating Request (NO registered letter),1524156387,SB11,DIRECTOR,12,12,59220,Thursday,1167,15438001,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1


## Deprecated Code

In [21]:
60*60*24

86400

### Pratice subprocess module

In [20]:
pickle_file = "random-a01.pkl"
result = subprocess.run(['scp', f'labnum08:git_repos/explainable-prescriptive-analytics/experiment_results/{pickle_file}', 'experiment_results/'], capture_output=True, text=True)

# return code of 0 means the command executed successfully
if result.returncode != 0:
    print("There is an Error in the command")
else:
    print("successful")

''

In [ ]:
def get_query_instance(sidx=14, eidx=16):
    assert eidx - sidx == 2, "One row represents the current action and the next one represents the suggested action"
    current_step = X_train[sidx: sidx+1]
    expected_next_step = X_train[eidx-1: eidx]
    return current_step, expected_next_step
# current_step, query_instances = get_query_instance(14, 16)

In [ ]:
def validate_transition(cfe):
    """ Deprecated function.
    Works with transition graph which has single activity as key.
    Args:
        cfe: Dice counterfactual object.
    """
    cf_examples_df = cfe.cf_examples_list[0].final_cfs_df.copy()

    # expected_next_action = cfe.cf_examples_list[0].test_instance_df[activity_column_name].item()
    # print(f"Expected next action: {expected_next_action}")

    current_activity = current_step[activity_column_name].item()
    # === Verify the next activity
    indexes_to_drop = []
    for idx, suggested_next_activity in cf_examples_df[activity_column_name].items():
        # print(f"index: {idx}, Suggest: {suggested_next_activity}")
        if suggested_next_activity not in transition_graph[current_activity]:
            indexes_to_drop.append(idx)

    cf_examples_df = cf_examples_df.drop(indexes_to_drop, axis='index').reset_index(drop=True)

    # === Verify the associated resources
    indexes_to_drop = []
    for idx, row in cf_examples_df[ ["ACTIVITY", "Involved_ST_Function_Div"] ].iterrows():
        row_tuple = tuple(row)
        if row_tuple not in valid_resources:
            # print(f"removed row had: {row_tuple}")
            indexes_to_drop.append(idx)

    cf_examples_df = cf_examples_df.drop(indexes_to_drop, axis='index').reset_index(drop=True)
    return cf_examples_df